# M3T1: Cast the columns of the orders_table to the correct data types
-- Change table column data type
alter table orders_table
alter column date_uuid type uuid using date_uuid::uuid,
alter column user_uuid type uuid using user_uuid::uuid,
alter column card_number type varchar(19),
alter column store_code type varchar(12),
alter column product_code type varchar(11),
alter column product_quantity type smallint;
select * from orders_table limit 5;


# Determine maximum length of the values in a column
SELECT MAX(LENGTH(card_number)) FROM orders_table;

# Check that column can be converted to uuid. Ensure no row is returned
SELECT date_uuid
FROM orders_table
WHERE NOT date_uuid ~* '^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}$';

# Postgres data types
https://www.postgresql.org/docs/current/datatype.html

# Run SQL querries in transaction mode
- Right click the database go to PSQL TOOL and run \set AUTOCOMMIT off.
- Run this to get the response \echo :AUTOCOMMIT
- Then when you want to start a transaction you can use BEGIN and COMMIT or ROLLBACK as needed.
i.e.
BEGIN; 
UPDATE accounts SET balance = balance + 100.00 WHERE acctnum = 12345; 
UPDATE accounts SET balance = balance - 100.00 WHERE acctnum = 7534; 
COMMIT; or ROLLBACK
- You can see all available psql command in the terminal with \? as well if you want to have a look through

# M3T2: Cast the columns of the dim_users to the correct data types
alter table dim_users
alter column first_name type varchar(255),
alter column last_name type varchar(255),
alter column date_of_birth type date,
alter column country_code type varchar(3),
alter column user_uuid type uuid using user_uuid::uuid,
alter column join_date type date;
select * from dim_users limit 5;

# M3T3: merge columns, change data types and update row
SELECT * FROM public.dim_store_details
BEGIN;
-- Step 1: Update the latitude column with values from the lat column where latitude is NULL
UPDATE dim_store_details
SET latitude = lat
WHERE latitude IS NULL;
-- Step 2: Drop the lat column
ALTER TABLE dim_store_details
DROP COLUMN lat;
COMMIT;

BEGIN;
UPDATE dim_store_details
SET longitude = NULL,
	locality = NULL,
	address = NULL,
	latitude = NULL
WHERE longitude = 'N/A' AND latitude = 'N/A' AND store_type = 'Web Portal';
COMMIT;

BEGIN;
alter table dim_store_details
alter column longitude type numeric using longitude::numeric,
alter column locality type varchar(255),
alter column store_code type varchar(12),
alter column staff_numbers type smallint USING staff_numbers::smallint,
alter column opening_date type date,
alter column store_type type varchar(255),
ALTER COLUMN store_type DROP NOT NULL,
alter column latitude type numeric using latitude::numeric,
alter column country_code type varchar(2),
alter column continent type varchar(255);
COMMIT;

# M3T4: Make changes to the dim_products table for the delivery team
BEGIN;
-- Remove £ from values in product_price column
UPDATE dim_products
SET product_price = REPLACE(product_price, '£', '');

-- Change datatype of product_type so this column can be used for calculation
ALTER TABLE dim_products
ALTER COLUMN product_price TYPE DOUBLE PRECISION USING product_price::DOUBLE PRECISION;

-- Add a new column called weight_class and populate it
-- Step 1: Add the new column weight_class
ALTER TABLE dim_products
ADD COLUMN weight_class VARCHAR(14);
-- Step 2: Update the weight_class column based on the weight range
UPDATE dim_products
SET weight_class = CASE
    WHEN weight < 2 THEN 'Light'
    WHEN weight >= 2 AND weight < 40 THEN 'Mid_Sized'
    WHEN weight >= 40 AND weight < 140 THEN 'Heavy'
    ELSE 'Truck_Required'
END;
COMMIT;

# M3T5: Update the dim_products table with the required data types
-- renaming column
BEGIN;
ALTER TABLE dim_products
RENAME COLUMN removed TO still_available;
select * from dim_products order by 1;
commit

-- determing varchar length
SELECT MAX(LENGTH("EAN")) FROM dim_products; -- 17
SELECT MAX(LENGTH(product_code)) FROM dim_products; -- 11

-- cleaning column
BEGIN;
UPDATE dim_products
SET still_available = CASE
    WHEN still_available = 'Still_avaliable' THEN 'TRUE'
    WHEN still_available = 'Removed' THEN 'FALSE'
END;
commit;

-- changing data type of columns
BEGIN;
alter table dim_products
alter column product_price type numeric using product_price::numeric,
alter column weight type numeric using weight::numeric,
alter column "EAN" type varchar(17),
alter column product_code type varchar(11),
alter column date_added type date,
alter column "uuid" type uuid using "uuid"::uuid,
alter column still_available type bool USING still_available::boolean;
-- weight_class has been created as varchar(14) in M3T4
commit;
select * from dim_products order by 1 limit 10;

# M3T6: Update the dim_date_times table
-- determing varchar length
SELECT MAX(LENGTH(time_period)) FROM dim_date_times; -- 10

-- changing data type of columns
BEGIN;
alter table dim_date_times
alter column month type varchar(2),
alter column year type varchar(4),
alter column day type varchar(2),
alter column time_period type varchar(10),
alter column date_uuid type uuid using date_uuid::uuid;
commit;
select * from dim_date_times limit 10;

# M3T7: Updating the dim_card_details table
-- determing varchar length
SELECT MAX(LENGTH(card_number)) FROM dim_card_details; -- 19
SELECT MAX(LENGTH(expiry_date)) FROM dim_card_details; -- 5

-- changing data type of columns
BEGIN;
alter table dim_card_details
alter column card_number type varchar(19),
alter column expiry_date type varchar(5),
alter column date_payment_confirmed type date;
commit;
select * from dim_card_details order by 1 limit 20;

# M3T8: Create the primary keys in the dimension tables
-- To create primary keys and update the respective columns in different tables, you need to follow these steps:
-- 1. Ensure that the columns you want to use as primary keys have unique values and do not contain NULL values.
SELECT product_code, COUNT(*)
FROM dim_products
GROUP BY product_code
HAVING COUNT(*) > 1;

SELECT *
FROM dim_products 
WHERE product_code IS NULL;

-- 2. Alter the table to add the primary key constraint. 
BEGIN;
ALTER TABLE dim_date_times
ADD CONSTRAINT dim_date_times_pkey PRIMARY KEY (date_uuid);
--do likewise for the other dim* tables
COMMIT;

# M3T9: Finalising the star-based schema by adding the foreign keys to the orders table
BEGIN;

ALTER TABLE orders_table
ADD CONSTRAINT fk_user_uuid FOREIGN KEY (user_uuid) REFERENCES dim_users(user_uuid);

ALTER TABLE orders_table
ADD CONSTRAINT fk_store_code FOREIGN KEY (store_code) REFERENCES dim_store_details(store_code);

ALTER TABLE orders_table
ADD CONSTRAINT fk_product_code FOREIGN KEY (product_code) REFERENCES dim_products(product_code);

ALTER TABLE orders_table
ADD CONSTRAINT fk_date_uuid FOREIGN KEY (date_uuid) REFERENCES dim_date_times(date_uuid);

ALTER TABLE orders_table
ADD CONSTRAINT fk_card_number FOREIGN KEY (card_number) REFERENCES dim_card_details(card_number);

COMMIT;

-- List the foreign keys in a table for a set of columns
SELECT
    tc.table_name,
    kcu.column_name,
    ccu.table_name AS foreign_table_name,
    ccu.column_name AS foreign_column_name
FROM
    information_schema.table_constraints AS tc
    JOIN information_schema.key_column_usage AS kcu
      ON tc.constraint_name = kcu.constraint_name
    JOIN information_schema.constraint_column_usage AS ccu
      ON ccu.constraint_name = tc.constraint_name
WHERE
    tc.constraint_type = 'FOREIGN KEY'
    AND kcu.table_name = 'orders_table'
    AND kcu.column_name IN ('user_uuid', 'store_code', 'product_code', 'date_uuid', 'card_number');

# M3T10: 